<a href="https://colab.research.google.com/github/ZahraShourmeij/Roberta/blob/Transformers-articles/preprocess_data_for_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install tokenizers pandas

In [4]:
!pip install transformers

In [5]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [7]:
from transformers import GPT2Tokenizer

# Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Read your dataset
df = pd.read_csv("EXIST2021_training.tsv", sep="\t")

# Tokenize and preprocess each text sample
preprocessed_texts = []
for text in df["text"]:
    # Tokenize the text
    tokens = tokenizer.encode(text)
    # Extract token IDs from each token
    token_ids = [token_id for token_id in tokens]
    # Append preprocessed token IDs to the preprocessed texts list
    preprocessed_texts.append(token_ids)

    # Save the preprocessed texts
with open("preprocessed_dataset.json", "w") as file:
    json.dump(preprocessed_texts, file)


In [8]:
# Save the preprocessed texts
with open("preprocessed_dataset.json", "w") as file:
    json.dump(preprocessed_texts, file)


In [ ]:
from transformers import RobertaTokenizer
import torch
from torch.utils.data import DataLoader, TensorDataset
import json

# Load RoBERTa tokenizer
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Load preprocessed token IDs from the JSON file
with open("preprocessed_dataset.json", "r") as file:
    preprocessed_texts = json.load(file)

# Pad or truncate sequences to a fixed length
max_length = 64  # adjust as needed
input_ids = [tokens[:max_length] + [roberta_tokenizer.pad_token_id] * (max_length - len(tokens)) for tokens in preprocessed_texts]

# Convert token IDs into tensors
input_ids = torch.tensor(input_ids)

# Create a TensorDataset
dataset = TensorDataset(input_ids)

# Define batch size and create DataLoader
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Load pre-trained RoBERTa model
roberta_model = RobertaForMaskedLM.from_pretrained('roberta-base')

# Define optimizer
optimizer = AdamW(roberta_model.parameters(), lr=5e-5)

# Define training loop
num_epochs = 3
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
roberta_model.to(device)

for epoch in range(num_epochs):
    roberta_model.train()
    for batch in dataloader:
        # Move batch to device
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "labels": batch[0]}  # Masked language modeling, predicting the same as input
        optimizer.zero_grad()
        outputs = roberta_model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the trained model
roberta_model.save_pretrained("roberta-trained-model")


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [ ]:
from transformers import RobertaForMaskedLM
from sklearn.metrics import classification_report

# Load the trained model
model = RobertaForMaskedLM.from_pretrained("path_to_trained_model")

# Load the validation or test dataset
# Replace `X_val` with your input features and `y_val` with corresponding labels
X_val, y_val = load_validation_data()

# Perform inference to obtain predictions
# Make sure to convert X_val to input tensors
inputs = {"input_ids": X_val}

# Forward pass through the model
with torch.no_grad():
    outputs = model(**inputs)

# Assuming classification task, convert logits to predicted labels
predicted_labels = torch.argmax(outputs.logits, dim=-1)

# Compute evaluation metrics
print(classification_report(y_val, predicted_labels))


In [ ]:
from transformers import RobertaForMaskedLM

# Load the trained RoBERTa model
model = RobertaForMaskedLM.from_pretrained("path_to_trained_model")

# Load or generate input data for inference
input_data = prepare_input_data("path_to_input_data")

# Perform inference
inputs = {"input_ids": input_data}

# Forward pass through the model
with torch.no_grad():
    outputs = model(**inputs)

# Process the outputs as needed
# For example, extract predicted tokens or scores
